In [1]:
import os
raiz = os.getcwd().replace('\\notebooks', '')
os.chdir(raiz)

In [ ]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
from secrets import uri
import pickle

In [2]:
# Obtengo los datos Últimos 12 datos
url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'
resp = requests.get(url)
data = pd.DataFrame(resp.json()['values'])

# Pongo bien la fecha
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

data.head()

,x,y
0,2016-06-11 00:00:00,1.983333
1,2016-06-11 00:15:00,1.966667
2,2016-06-11 00:30:00,1.900000
3,2016-06-11 00:45:00,2.283333
4,2016-06-11 01:00:00,2.083333


In [3]:
# Leo la última predicción realizada
engine = create_engine(uri)
query = engine.execute('SELECT MAX(fecha_realidad) FROM realidad;')
ultima_fecha_realidad = query.fetchall()[0][0]
query.close()

# Leo la última predicción realizada
engine = create_engine(uri)
query = engine.execute('SELECT MIN(fecha_prediccion), MAX(fecha_prediccion) FROM predicciones;')
fecha_prediccion = query.fetchall()[0]
query.close()


primera_fecha_prediccion = fecha_prediccion[0]
ultima_fecha_prediccion = fecha_prediccion[1]

print(ultima_fecha_realidad)
print(primera_fecha_prediccion)
print(ultima_fecha_prediccion)


None
2021-11-25 10:00:00
2021-12-02 09:00:00


In [4]:
if ultima_fecha_realidad is None:
    fecha_extraer = primera_fecha_prediccion

elif  ultima_fecha_realidad <= ultima_fecha_prediccion:
    fecha_extraer = ultima_fecha_realidad

else:
    fecha_extraer = ultima_fecha_realidad

print(fecha_extraer)

2021-11-25 10:00:00


In [18]:
# Reondeo las horas
data['x'] = data['x'].dt.round('H')

# Obtengo nº de transacciones por hora
data_grouped = data\
  .groupby('x')\
  .sum()\
  .reset_index()

# Me quedo con los datos a partir de esa fecha
data_grouped = data_grouped.loc[data_grouped['x'] >= fecha_extraer,:]

data_grouped.head()


,x,y
47817,2021-11-25 10:00:00,17.933333
47818,2021-11-25 11:00:00,9.566667
47819,2021-11-25 12:00:00,17.416667
47820,2021-11-25 13:00:00,11.216667
47821,2021-11-25 14:00:00,22.150000


In [21]:
# Subo los datos reales a la bbdd
upload_data = list(zip(data_grouped['x'], round(data_grouped['y'],4)))
upload_data[:3]

[(Timestamp('2021-11-25 10:00:00'), 17.9333),
 (Timestamp('2021-11-25 11:00:00'), 9.5667),
 (Timestamp('2021-11-25 12:00:00'), 17.4167)]

In [25]:
# Hago el insert
for upload_dia in upload_data:
    timestamp, realidad = upload_dia

    result = engine.execute(f"INSERT INTO realidad (fecha_realidad, realidad)\
        VALUES('{timestamp}', '{realidad}') \
        ON CONFLICT (fecha_realidad) DO UPDATE \
        SET fecha_realidad = '{timestamp}', \
            realidad = '{realidad}'\
        ;")
    result.close()
